# Week 10 Assignment

Because I was unable to conduct our workshop this week, I'm keeping the assignment light as well.  Below you'll find just two steps for this week: one programming exercise and then a planning activity for your final project.

For clarification, the "final project" I've been referring to is your "final."  It is not a project in addition to a final exam.  They're one-in-the-same.

Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `/data/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
# This is just to show you the name to use for the variable you need to create for this step to pass.
hospitals = pd.read_csv('./complications_all.csv')
mo_hospitals=hospitals[(hospitals['State']=='MO')]
mo_hospitals

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Start Date,End Date
45534,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,No Different Than the National Rate,26,2.5,1.4,4.2,NaN,04/01/2015,03/31/2018
45535,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_AMI,Death rate for heart attack patients,No Different Than the National Rate,175,13.9,11.0,16.9,NaN,07/01/2015,06/30/2018
45536,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_CABG,Death rate for CABG surgery patients,No Different Than the National Rate,91,2.5,1.2,5.1,NaN,07/01/2015,06/30/2018
45537,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_COPD,Death rate for COPD patients,No Different Than the National Rate,326,8.5,6.5,10.9,NaN,07/01/2015,06/30/2018
45538,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_HF,Death rate for heart failure patients,No Different Than the National Rate,461,13.1,10.7,15.9,NaN,07/01/2015,06/30/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47662,263304,SHRINERS HOSPITALS FOR CHILDREN,4400 CLAYTON AVE,SAINT LOUIS,MO,63110,ST. LOUIS CITY,(314) 432-3600,PSI_4_SURG_COMP,Deaths among Patients with Serious Treatable C...,Not Available,Not Available,Not Available,Not Available,Not Available,19.0,07/01/2016,06/30/2018
47663,263304,SHRINERS HOSPITALS FOR CHILDREN,4400 CLAYTON AVE,SAINT LOUIS,MO,63110,ST. LOUIS CITY,(314) 432-3600,PSI_6_IAT_PTX,Collapsed lung due to medical treatment,Not Available,Not Available,Not Available,Not Available,Not Available,19.0,07/01/2016,06/30/2018
47664,263304,SHRINERS HOSPITALS FOR CHILDREN,4400 CLAYTON AVE,SAINT LOUIS,MO,63110,ST. LOUIS CITY,(314) 432-3600,PSI_8_POST_HIP,Broken hip from a fall after surgery,Not Available,Not Available,Not Available,Not Available,Not Available,19.0,07/01/2016,06/30/2018
47665,263304,SHRINERS HOSPITALS FOR CHILDREN,4400 CLAYTON AVE,SAINT LOUIS,MO,63110,ST. LOUIS CITY,(314) 432-3600,PSI_90_SAFETY,Serious complications,Not Available,Not Available,Not Available,Not Available,Not Available,19.0,07/01/2016,06/30/2018


In [3]:
# These assertions will help make sure that you're on the right track.
assert(mo_hospitals['State'].unique() == ['MO'])
assert(mo_hospitals.shape == (2133,18))

In [4]:
import numpy as np
from numpy import nan as NaN
from pandas import Series,DataFrame
mo_hospitals['Start Date'] = mo_hospitals['Start Date'].astype('datetime64[ns]')
mo_hospitals['End Date'] = mo_hospitals['End Date'].astype('datetime64[ns]')

mo_hospitals=mo_hospitals[~mo_hospitals['Denominator'].isin(['Not Available'])]
mo_hospitals_sort=mo_hospitals.drop(mo_hospitals.columns[[0,2,3,4,5,6,7,9,10,12,13,14,15]],axis=1)

mo_hospitals_d=mo_hospitals_sort.pivot(index='Facility Name', columns='Measure ID', values='Denominator')
mo_hospitals_d1=mo_hospitals_d.fillna(0)
mo_hospitals_d1['COMP_HIP_KNEE']=mo_hospitals_d1['COMP_HIP_KNEE'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['MORT_30_AMI']=mo_hospitals_d1['MORT_30_AMI'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['MORT_30_CABG']=mo_hospitals_d1['MORT_30_CABG'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['MORT_30_COPD']=mo_hospitals_d1['MORT_30_COPD'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['MORT_30_HF']=mo_hospitals_d1['MORT_30_HF'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['MORT_30_PN']=mo_hospitals_d1['MORT_30_PN'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['MORT_30_STK']=mo_hospitals_d1['MORT_30_STK'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_10_POST_KIDNEY']=mo_hospitals_d1['PSI_10_POST_KIDNEY'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_11_POST_RESP']=mo_hospitals_d1['PSI_11_POST_RESP'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_12_POSTOP_PULMEMB_DVT']=mo_hospitals_d1['PSI_12_POSTOP_PULMEMB_DVT'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_13_POST_SEPSIS']=mo_hospitals_d1['PSI_13_POST_SEPSIS'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_14_POSTOP_DEHIS']=mo_hospitals_d1['PSI_14_POSTOP_DEHIS'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_15_ACC_LAC']=mo_hospitals_d1['PSI_15_ACC_LAC'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_3_ULCER']=mo_hospitals_d1['PSI_3_ULCER'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_4_SURG_COMP']=mo_hospitals_d1['PSI_4_SURG_COMP'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_6_IAT_PTX']=mo_hospitals_d1['PSI_6_IAT_PTX'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_8_POST_HIP']=mo_hospitals_d1['PSI_8_POST_HIP'].str.replace("`","").astype(np.float64)
mo_hospitals_d1['PSI_9_POST_HEM']=mo_hospitals_d1['PSI_9_POST_HEM'].str.replace("`","").astype(np.float64)

mo_hospitals_d1['number'] = mo_hospitals_d1.sum(axis=1)
mo_hospitals_d1.head()
mo_summary=mo_hospitals_d1.drop(mo_hospitals_d1.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]],axis=1)


mo_summary.head()

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Measure ID,number
Facility Name,
BARNES JEWISH HOSPITAL,131313.0
BARNES-JEWISH ST PETERS HOSPITAL,15668.0
BARNES-JEWISH WEST COUNTY HOSPITAL,9622.0
BATES COUNTY MEMORIAL HOSPITAL,3117.0
BELTON REGIONAL MEDICAL CENTER,9270.0


In [6]:
mo_summary_s=mo_hospitals_sort.groupby(by='Facility Name').min('Start Date')
mo_summary_e=mo_hospitals_sort.groupby(by='Facility Name').min('End Date')
mo_summary['start_date'] = mo_summary_s['Start Date'].values
mo_summary['end_date'] = mo_summary_e['End Date'].values
mo_summary.head()

Measure ID,number,start_date,end_date
Facility Name,,,
BARNES JEWISH HOSPITAL,131313.0,2015-04-01,2018-03-31
BARNES-JEWISH ST PETERS HOSPITAL,15668.0,2015-04-01,2018-03-31
BARNES-JEWISH WEST COUNTY HOSPITAL,9622.0,2015-04-01,2018-03-31
BATES COUNTY MEMORIAL HOSPITAL,3117.0,2015-07-01,2018-06-30
BELTON REGIONAL MEDICAL CENTER,9270.0,2015-04-01,2018-03-31


In [4]:
assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

---

## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Save this note with Ctrl-S (or Cmd-S)
2. Skip down to the last command cell (the one starting with `%%bash`) and run that cell.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

In [ ]:
assert False, "DO NOT REMOVE THIS LINE"

---

In [64]:
%%bash
git pull
git add week10_assignment_2.ipynb
git commit -a -m "Submitting the week 10 programming assignment"
git push

Already up to date.
[main e4cb6ac] Submitting the week 10 programming assignment
 2 files changed, 1305 insertions(+), 71 deletions(-)
 create mode 100644 week10/week10_assignment_2.ipynb


remote: This repository moved. Please use the new location:        
remote:   git@github.com:Xiaoniu86/hds5210-2021.git        
To github.com:xiaoniu86/hds5210-2021.git
   5499ee3..e4cb6ac  main -> main



---

If the message above says something like _Submitting the week 8 programming assignment_ or _Everything is up to date_, then your work was submitted correctly.